In [15]:
import json
from pathlib import Path
import shutil
from collections import Counter

In [16]:
from tqdm import tqdm

In [17]:
input_json_pose_folder = '../data/wiki_crop_mtcnn_pose_detection/json'
input_json_filter_file = '../data/good_data3.json'

sample_parent = '../data/wiki_crop/'
sample_output = '../data/wiki_crop_filtered_pose'

In [18]:
all_records = {}
for json_file in Path(input_json_pose_folder).glob('*.json'):
    with open(json_file, 'r') as f:
        data = json.load(f)
        for key, value in data.items():
            all_records[f'{json_file.stem}/{key}'] = value

In [19]:
with open(input_json_filter_file, 'r') as f:
    good_data = json.load(f)

In [20]:
len(all_records)

62328

In [21]:
filtered_count = Counter()
filtered_records = {}

for key, value in tqdm(all_records.items()):
    if len(value) > 1:
        filtered_count['multiple'] += 1
        continue
    
    value = value[0]
    
    if value['error'] is not None:
        filtered_count[f'{value["error"]}: {value["error_detail"]}'] += 1
        continue
        
    if key[3:] not in good_data:
        filtered_count['bad_data'] += 1
        continue
        
    if value['pred_label'] != 'frontal':
        filtered_count['not_frontal'] += 1
        continue
    
    filtered_records[key] = value

100%|██████████| 62328/62328 [00:24<00:00, 2563.86it/s]


In [22]:
filtered_count

Counter({'not_frontal': 25189,
         'error_runtime: torch.cat(): expected a non-empty list of Tensors': 8070,
         'error_no_face_detected: Bounding box is None, no face detected': 5452,
         'bad_data': 5177,
         'multiple': 3089})

In [23]:
len(filtered_records)

15351

In [25]:
with open('../data/filtered_pose.json', 'w', encoding='utf-8') as f:
    json.dump({k[3:]: v for k, v in filtered_records.items()}, f, ensure_ascii=False, indent=4)

In [26]:
Path(sample_output).mkdir(exist_ok=True)

for file_name in tqdm(filtered_records):
    shutil.copy(str(Path(sample_parent) / file_name), str(Path(sample_output) / file_name[3:]))

100%|██████████| 15351/15351 [00:45<00:00, 334.23it/s]
